In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline
data=pd.read_excel('data/Data_Set_(A+B).xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'data/Data_Set_(A+B).xlsx'

In [1]:
data

NameError: name 'data' is not defined

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   I1      176 non-null    float64
 1   I2      176 non-null    float64
 2   I3      176 non-null    float64
 3   gx      176 non-null    float64
 4   gy      176 non-null    float64
 5   gz      176 non-null    float64
 6   ax      176 non-null    float64
 7   ay      176 non-null    float64
 8   az      176 non-null    float64
 9   V1real  176 non-null    float64
 10  V2real  176 non-null    float64
 11  V3real  176 non-null    float64
 12  N1      176 non-null    int64  
 13  N2      176 non-null    int64  
 14  N3      176 non-null    int64  
 15  Type    176 non-null    int64  
dtypes: float64(12), int64(4)
memory usage: 22.1 KB


In [ ]:
R = 0.04      # 轮子半径，单位：米（40 mm）
L = 0.125     # 轮子到机器人中心的距离（125 mm）
alpha = np.deg2rad(0)  # α：机器人朝向角（单位：弧度）
theta = np.deg2rad(30)  # Θ：轮子朝向角（单位：弧度）

In [ ]:
from numpy import cos, sin

# 3x3 矩阵乘以 3x1 向量
M = np.array([
    [(-2/3)*cos(alpha - theta), (2/3)*sin(alpha),       (2/3)*cos(alpha + theta)],
    [(-2/3)*sin(alpha - theta), (-2/3)*cos(alpha),      (2/3)*sin(alpha + theta)],
    [1/(3*L),                   1/(3*L),                1/(3*L)]
])

In [ ]:
def compute_velocities(row):
    omega = np.array([row['V1real'], row['V2real'], row['V3real']])
    result = R * (M @ omega)  # R × M × ω 向量
    return pd.Series(result, index=['Vx', 'Vy', 'Omega'])

data[['Vx', 'Vy', 'Omega']] = data.apply(compute_velocities, axis=1)


In [ ]:
M_current = np.array([
    [(-2/3)*np.cos(alpha - theta), (2/3)*np.sin(alpha),       (2/3)*np.cos(alpha + theta)],
    [(-2/3)*np.sin(alpha - theta), (-2/3)*np.cos(alpha),      (2/3)*np.sin(alpha + theta)],
    [1/3,                          1/3,                       1/3]
])
# 计算 Ix, Iy, Iphi（逐行应用矩阵乘法）
def compute_projected_currents(row):
    I = np.array([row['I1'], row['I2'], row['I3']])
    result = M_current @ I
    return pd.Series(result, index=['Ix', 'Iy', 'Iphi'])
# IΣ（电流总和）
data['Isigma'] = data['I1'] + data['I2'] + data['I3']


data[['Ix', 'Iy', 'Iphi']] = data.apply(compute_projected_currents, axis=1)

In [ ]:
data

,I1,I2,I3,gx,gy,gz,ax,ay,az,V1real,...,N2,N3,Type,Vx,Vy,Omega,Ix,Iy,Iphi,Isigma
0,1.126697,0.021116,0.927601,0.712941,-0.252941,1.081765,0.043529,0.003174,0.011661,-249.352941,...,30,5239,1,11.883906,0.084706,2.202353,-0.114948,0.670689,0.691805,2.075414
1,1.039215,0.015083,0.953243,0.229412,0.276471,0.485294,0.013672,-0.005572,0.012164,-260.470588,...,-9,5391,1,12.103979,0.042353,0.338824,-0.049636,0.654097,0.669180,2.007541
2,0.911011,0.004525,0.947210,1.170588,0.222941,0.664118,0.011704,0.003720,0.005744,-268.411765,...,6,4786,1,12.397408,0.000000,0.000000,0.020900,0.616390,0.620915,1.862746
3,0.692308,1.156862,0.550528,1.165294,0.698235,-3.421765,0.001321,-0.010311,-0.001522,141.352941,...,-5662,2875,1,0.476823,12.218824,-0.338824,-0.081857,-0.356963,0.799899,2.399698
4,0.529412,1.024133,0.453997,0.808235,0.090000,-1.449412,0.006218,-0.010986,-0.006247,155.647059,...,-5824,2967,1,0.330109,12.727059,1.185882,-0.043541,-0.354953,0.669181,2.007542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,0.288889,0.449573,0.270085,0.153333,-2.288667,40.140000,-0.019906,0.003939,-0.000586,349.200000,...,5663,5901,5,0.207846,0.360000,111.744000,-0.010856,-0.113390,0.336182,1.008547
172,0.314530,0.331624,0.341880,0.096000,-2.154667,37.425333,-0.015479,0.000993,-0.009017,347.400000,...,6018,6005,5,0.290985,0.264000,112.128000,0.015791,-0.002279,0.329345,0.988034
173,0.287179,0.377778,0.391453,-1.820667,-5.030000,82.963333,0.027490,0.046403,-0.012874,750.600000,...,12073,12049,5,-0.290985,0.120000,237.696000,0.060202,-0.025641,0.352137,1.056410
174,0.370940,0.427350,0.336752,-0.624000,-4.574000,83.285333,0.014160,-0.043896,0.003906,734.400000,...,12493,12564,5,0.290985,-0.120000,237.504000,-0.019738,-0.049003,0.378347,1.135042


In [ ]:
# 为防止除以 0，先加一个小常数 epsilon
epsilon = 1e-6

# 计算 {V3}
data['Tx'] = data['Vx'] / (data['Ix'] + epsilon)
data['Ty'] = data['Vy'] / (data['Iy'] + epsilon)
data['Tphi'] = data['Omega'] / (data['Iphi'] + epsilon)
data['Tz'] = data['gz'] / (data['Iphi'] + epsilon)


In [ ]:
data

,I1,I2,I3,gx,gy,gz,ax,ay,az,V1real,...,Vy,Omega,Ix,Iy,Iphi,Isigma,Tx,Ty,Tphi,Tz
0,1.126697,0.021116,0.927601,0.712941,-0.252941,1.081765,0.043529,0.003174,0.011661,-249.352941,...,0.084706,2.202353,-0.114948,0.670689,0.691805,2.075414,-103.385877,0.126297,3.183485,1.563683
1,1.039215,0.015083,0.953243,0.229412,0.276471,0.485294,0.013672,-0.005572,0.012164,-260.470588,...,0.042353,0.338824,-0.049636,0.654097,0.669180,2.007541,-243.859123,0.064750,0.506325,0.725206
2,0.911011,0.004525,0.947210,1.170588,0.222941,0.664118,0.011704,0.003720,0.005744,-268.411765,...,0.000000,0.000000,0.020900,0.616390,0.620915,1.862746,593.161248,0.000000,0.000000,1.069577
3,0.692308,1.156862,0.550528,1.165294,0.698235,-3.421765,0.001321,-0.010311,-0.001522,141.352941,...,12.218824,-0.338824,-0.081857,-0.356963,0.799899,2.399698,-5.825169,-34.230064,-0.423582,-4.277739
4,0.529412,1.024133,0.453997,0.808235,0.090000,-1.449412,0.006218,-0.010986,-0.006247,155.647059,...,12.727059,1.185882,-0.043541,-0.354953,0.669181,2.007542,-7.581752,-35.855742,1.772138,-2.165946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,0.288889,0.449573,0.270085,0.153333,-2.288667,40.140000,-0.019906,0.003939,-0.000586,349.200000,...,0.360000,111.744000,-0.010856,-0.113390,0.336182,1.008547,-19.147176,-3.174901,332.390217,119.399192
172,0.314530,0.331624,0.341880,0.096000,-2.154667,37.425333,-0.015479,0.000993,-0.009017,347.400000,...,0.264000,112.128000,0.015791,-0.002279,0.329345,0.988034,18.426266,-115.883332,340.457044,113.635473
173,0.287179,0.377778,0.391453,-1.820667,-5.030000,82.963333,0.027490,0.046403,-0.012874,750.600000,...,0.120000,237.696000,0.060202,-0.025641,0.352137,1.056410,-4.833364,-4.680160,675.008766,235.599157
174,0.370940,0.427350,0.336752,-0.624000,-4.574000,83.285333,0.014160,-0.043896,0.003906,734.400000,...,-0.120000,237.504000,-0.019738,-0.049003,0.378347,1.135042,-14.742788,2.448889,627.738800,220.128651


In [ ]:
# 保存为 Excel 文件，index=False 表示不保存行号
data.to_excel('data/processed_(A+B)_data.xlsx', index=False, engine='openpyxl')
